In [ ]:
import pysam
import numpy as np
import pandas as pd

In [ ]:
bams = ! gsutil ls gs://ccle_bams/**.bam

In [ ]:
bams

In [ ]:
res = {}
i = 0

In [ ]:
for i, bam in enumerate(bams[i:]):
    print(i/len(bams[i:]),end='\r')
    a = pysam.AlignmentFile('data/seqs/'+'/'.join(bam.split('/')[3:]), mode="r",threads=4)
    try:
        dt = [val['DT'].split('T')[0] for val in a.header['RG']]
    except KeyError:
        res[bam] = "unknown"
        continue
    if len(dt) >1:
        arr=np.array(dt[0].split('-')).astype(int)
        for val in dt[1:]:
            arr = np.vstack((arr,np.array(val.split('-')).astype(int)))
        arr = arr.T
        i = arr[0]*365 + arr[1] * 30 + arr[2]
        res[bam] = dt[np.argsort(i)[0]]
    else:
        res[bam] = dt[0]

In [ ]:
import json
with open('saved.json', 'w') as json_file:
    json.dump(res, json_file)

In [ ]:
i = 4209

In [ ]:
len(bams)

In [ ]:
res

In [ ]:
def maxdate(dt):
    """
    gets the maxdate from a list of str date formated YYYY-MM-DD
    """
    arr=np.array(dt[0].split('-')).astype(int)
    if len(dt) >1:
        for val in dt[1:]:
            arr = np.vstack((arr,np.array(dt[0].split('-')).astype(int))).T
        i = arr[0]*365 + arr[1] * 30 + arr[2]
        
        return dt[np.argsort(i)[0]]
    else:
        return dt[0]